In [1]:
import pandas as pd
import numpy as np
import ccxt  #라이브러리 임포트
import time
import datetime as dt
from datetime import datetime
import time

# Binance API data

In [2]:
# 코인 정보 가져오기(한달 기준)

binance = ccxt.binanceusdm()
coin_name = 'BTC'  #코인명 입력
today = '20221130'  #오늘 날짜 입력
time_ = '1h'  #바이낸스에서 불러올 데이터 시간 단위 설정
              #  1분 : 1m 
              #  5분 : 3m
              #  15분 : 15m
              #  30분 : 30m
              #  1시간 : 1h
              #  6시간 : 6h
              #  12시간 : 12h
              #  1일 : 1d
              #  3일 : 3d
              #  1주일 : 1w
              #  1달 : 1M

# fetchOHLCV 매서드 ver.2
# fetchOHLCV ('BTC/USDT', timeframe = '1m', since = 'undefined', limit = 'undefined', params = {})
dateSet = []
def load_coin_data(symbol, time_m, date) : 
    dateSet.append(date)
    date = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')  #타입 변환
    dt_utc = date + dt.timedelta(hours=-9)  #KST -> UTC

    dt_utc = dt_utc.strftime('%Y-%m-%d %H:%M:%S')  #datetime -> str
    timestamp = time.mktime(datetime.strptime(dt_utc, '%Y-%m-%d %H:%M:%S').timetuple())*1000  #stamp형태로 변환

    BTC_data = binance.fetch_ohlcv(symbol, time_m, since = round(timestamp))  #코인명, 코인시세 간격 입력
    df_BTC = pd.DataFrame(BTC_data, columns = ['datetime','Open','High','Low','Close','Volume'])
    
    df_BTC['datetime'] = pd.to_datetime(df_BTC['datetime'], unit='ms')
    
    for i in range(len(df_BTC)) :
        df_BTC.loc[i,'datetime'] = df_BTC.loc[i,'datetime'] + dt.timedelta(hours=9)  #UTC -> KST

    return df_BTC

In [3]:
bcoin_name = coin_name + '/USDT'
date_1 = today[0:4]+'-'+today[4:6]+'-01 00:00:00'
if int(today[6:9]) >= 15 :
    date_2 = today[0:4]+'-'+today[4:6]+'-15 00:00:00'
else :
    date_2 = today[0:4]+'-'+today[4:6]+'-'+today[6:9]+' 00:00:00'
date_2

#바이낸스 데이터 전처리
coin_data1 = load_coin_data(bcoin_name, time_, date_1)
coin_data2 = load_coin_data(bcoin_name, time_, date_2)
print(coin_data1.head(3))
print(coin_data2.tail(3))
binanace_coin_20221226 = pd.concat([coin_data1, coin_data2], axis = 0)
binanace_coin_20221226 = binanace_coin_20221226.drop_duplicates(['datetime'])
binanace_coin_20221226 = binanace_coin_20221226.astype({'datetime' : 'str'})
binanace_coin_20221226['Date'] = binanace_coin_20221226['datetime'].str.contains(f'{today[0:4]}-{today[4:6]}')
binanace_coin_20221226 = binanace_coin_20221226[(binanace_coin_20221226['Date'] == True)]
binanace_coin_20221226 = binanace_coin_20221226.drop(["Date"], axis = 1)
binanace_coin_20221226 = binanace_coin_20221226.reset_index(drop=True)
binanace_coin_20221226

             datetime     Open     High      Low    Close     Volume
0 2022-10-31 15:00:00  20535.4  20555.1  20445.2  20507.2  14243.266
1 2022-10-31 16:00:00  20507.2  20539.6  20451.7  20537.2   7592.754
2 2022-10-31 17:00:00  20537.2  20537.2  20441.4  20478.8  12142.348
               datetime     Open     High      Low    Close     Volume
497 2022-12-05 08:00:00  17094.6  17206.2  17072.7  17099.5  20413.221
498 2022-12-05 09:00:00  17099.5  17259.8  17068.0  17205.8  34217.987
499 2022-12-05 10:00:00  17205.8  17282.7  17187.7  17256.9  16254.137


,datetime,Open,High,Low,Close,Volume
0,2022-11-01 00:00:00,20326.5,20448.6,20276.0,20390.4,27941.596
1,2022-11-01 01:00:00,20390.5,20523.5,20318.5,20366.3,36726.458
2,2022-11-01 02:00:00,20366.2,20414.0,20335.4,20363.3,12709.912
3,2022-11-01 03:00:00,20363.3,20460.0,20346.1,20424.2,10758.756
4,2022-11-01 04:00:00,20424.1,20426.9,20350.1,20358.8,11225.242
...,...,...,...,...,...,...
715,2022-11-30 19:00:00,16870.7,16891.7,16858.4,16858.4,6739.155
716,2022-11-30 20:00:00,16858.4,16891.9,16858.0,16871.4,7309.810
717,2022-11-30 21:00:00,16871.3,16879.5,16750.8,16771.2,18226.893
718,2022-11-30 22:00:00,16771.2,16888.5,16761.3,16802.9,37337.300


# OneBot API data

In [5]:
import requests
import json

ocoin_name = coin_name + 'USDT'

# Request
requestData = requests.get(' URL ')
print(requestData)

if requestData.status_code == 200 :
    print('Request OK')
    jsonData = requestData.json()

# 원봇 데이터 전처리
onebot_coin_score_20221226 = pd.DataFrame(jsonData)
onebot_coin_score_20221226 = onebot_coin_score_20221226.drop(["info"], axis = 1)
onebot_coin_score_20221226 = onebot_coin_score_20221226.transpose()
onebot_coin_score_20221226 = onebot_coin_score_20221226[['date', 'score']]
onebot_coin_score_20221226['Date'] = onebot_coin_score_20221226['date'].str.contains(f'{today[0:4]}-{today[4:6]}')
onebot_coin_score_20221226 = onebot_coin_score_20221226[(onebot_coin_score_20221226['Date'] == True)]
onebot_coin_score_20221226 = onebot_coin_score_20221226.drop(["Date"], axis = 1)
onebot_coin_score_20221226 = onebot_coin_score_20221226.reset_index(drop=True)

# 데이터 합치기
coin_data_20221226 = pd.concat([binanace_coin_20221226, onebot_coin_score_20221226], axis = 1)
coin_data_20221226 = coin_data_20221226.drop(["date"], axis = 1)

coin_data_20221226

<Response [200]>
Request OK


,datetime,Open,High,Low,Close,Volume,score
0,2022-11-01 00:00:00,20326.5,20448.6,20276.0,20390.4,27941.596,218.1
1,2022-11-01 01:00:00,20390.5,20523.5,20318.5,20366.3,36726.458,199.7
2,2022-11-01 02:00:00,20366.2,20414.0,20335.4,20363.3,12709.912,214
3,2022-11-01 03:00:00,20363.3,20460.0,20346.1,20424.2,10758.756,188.5
4,2022-11-01 04:00:00,20424.1,20426.9,20350.1,20358.8,11225.242,186.2
...,...,...,...,...,...,...,...
715,2022-11-30 19:00:00,16870.7,16891.7,16858.4,16858.4,6739.155,NaN
716,2022-11-30 20:00:00,16858.4,16891.9,16858.0,16871.4,7309.810,NaN
717,2022-11-30 21:00:00,16871.3,16879.5,16750.8,16771.2,18226.893,NaN
718,2022-11-30 22:00:00,16771.2,16888.5,16761.3,16802.9,37337.300,NaN


In [7]:
# 데이터 합치기
coin_data_20221226 = pd.concat([binanace_coin_20221226, onebot_coin_score_20221226], axis = 1)
coin_data_20221226 = coin_data_20221226.drop(["date"], axis = 1)
# total_data_20221226 = total_data_20221226.drop(["date"], axis = 1) # 인덱스 넘버로 행 삭제
print(coin_data_20221226)

# 파일 저장
from openpyxl import Workbook
file_name = coin_name + '_data_' + today + '.xlsx'
coin_data_20221226.to_excel(f'{file_name}') #excel파일로 생성

                datetime     Open     High      Low    Close     Volume  score
0    2022-11-01 00:00:00  20326.5  20448.6  20276.0  20390.4  27941.596  218.1
1    2022-11-01 01:00:00  20390.5  20523.5  20318.5  20366.3  36726.458  199.7
2    2022-11-01 02:00:00  20366.2  20414.0  20335.4  20363.3  12709.912    214
3    2022-11-01 03:00:00  20363.3  20460.0  20346.1  20424.2  10758.756  188.5
4    2022-11-01 04:00:00  20424.1  20426.9  20350.1  20358.8  11225.242  186.2
..                   ...      ...      ...      ...      ...        ...    ...
715  2022-11-30 19:00:00  16870.7  16891.7  16858.4  16858.4   6739.155    NaN
716  2022-11-30 20:00:00  16858.4  16891.9  16858.0  16871.4   7309.810    NaN
717  2022-11-30 21:00:00  16871.3  16879.5  16750.8  16771.2  18226.893    NaN
718  2022-11-30 22:00:00  16771.2  16888.5  16761.3  16802.9  37337.300    NaN
719  2022-11-30 23:00:00  16802.9  16878.0  16798.6  16838.5  15293.015    NaN

[720 rows x 7 columns]
